# MgO Analysis Manual Scans

We start with manual scans from MgO 1800 nm short pulses

In [ ]:
## necessary import statements
from HARP.scan_anal import Scan
from HARP.iris_functions import Iris
from HARP.image_processing import Image
import os
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, savgol_filter, peak_widths
import numpy as np 
import matplotlib.pyplot as plt
from functions import Andor_calib
from functions import Adams_wedge_thickness, gaus
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [ ]:
power = np.array([780 ,745 ,635 ,490 ,300 ,95 ,10 ,10.5 ,14 ,20 ,26 ,33 ,41 ,50 ,59 ,69 ,80 ,10
,6 ,3 ,1.2 ,0 ,470])*1e-3
iris_pos = np.array([45,30,15	,0,-15,-30,-40,-39,-38,-37,-36,-35,-34,-33,-32,-31,-30,-40,-41,-42,-43,-44,0])


iris_27 = Iris()
iris_27.specify_calib(iris_positions=iris_pos, powers=power)
iris_27.specify_params(w0_init=100e-6, f=0.75, wl=800e-9, M2=1, reprate=1000,pulse_duration=15e-15)


In [ ]:
data_folder = '/Volumes/qolslc/20120009 - Matthews' 
data_folder = os.path.join(data_folder,'2022-01-27')
os.listdir(data_folder)

In [ ]:
l = os.listdir(os.path.join(data_folder,'Rotation_Scan_MgO_30000trans_min22deg'))
[int(i.split('d')[0]) for i in l]

# Trans 30000
Manual Scans but this is the data we have from the MgO

In [ ]:
I40= os.path.join(data_folder,'Iris_Scan_MgO_800_trans40000')
I70= os.path.join(data_folder,'Iris_Scan_MgO_800_trans70000')
I90= os.path.join(data_folder,'Iris_Scan_MgO_800_trans90000')

In [ ]:
bob4 = Scan()
bob4.set_folder(I40)
bob4.set_verlim(0, 5000)
files = [i for i in os.listdir(I90)]
bg = []
exclude = bg+[]
files = [i for i in files if i not in exclude]
print(files)

In [ ]:
files[1].split('d')

In [ ]:
variables = [-1*float(i.split('d')[0].split('n')[1]) for i in files]
bob4.set_params(wedge=1060, MCPPos=40000,iris=-22)
bob4.set_eVlim((4,35))
stage = 'intensity'
variables.sort()

print(variables)

In [ ]:
bob4.populate_scan_manual(files, variables, stage='iris', function = Andor_calib)
bob4.add_calibration_stage('intensity', iris_27.get_intensity_TWcm2, 'iris')
#bob4.substract_bg(byitself=True, bg_lim=[0,0,10,10])
bob4.sort_by_stage('iris')

In [ ]:
bob4.add_calibration_stage('intensity', iris_27.get_intensity_TWcm2, 'iris')

In [ ]:
fig, ax = plt.subplots(1)
im, ax, ax2 = bob4.plot_average(ax)
ax.set_title('Average Trace')
ax.xaxis.tick_top()
ax.set_xlabel('Energy [eV]', va = 'top')
ax.set_ylabel('Divergence [a.u.]')


In [ ]:
bob4.scan_data['Data'].values[-1].data.shape

In [ ]:
#plt.contourf(bob4.scan_data['Data'].values[-1].eV_axis, np.arange(400), bob4.scan_data['Data'].values[-1].data, vmax = 300, levels=50)
plt.imshow(np.log(bob4.scan_data['Data'].values[-1].data),vmax=5.5,aspect='auto')

plt.ylim(150,300)
plt.colorbar()

In [ ]:
x, y, Z = bob4.return_scan_data_slab('iris')
fig, ax = plt.subplots(figsize = (20,10))
#cma = ax.contourf(y,x,np.log10(Z), levels=50, cmap='ocean_r')
cma = ax.imshow(Z, cmap='ocean_r', extent=[max(y), min(y), -1,1], aspect='auto', vmax=400)
cba = plt.colorbar(cma, ax=ax)
ax.set_xlim(1240/17, 1240/24)
ax.set_yticks([])